# NITROGEN ACCOUNTANCY IN SPACE AGRICULTURE
## NP fitting for all crops to MEC output
---

In [ ]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

from DU4 import *
import time

import matplotlib.style as style
plt.rcParams.update(plt.rcParamsDefault)
style.use('seaborn-v0_8-poster')
plt.rcParams['figure.dpi'] = 300

'''
Spines & lines
'''
box_lw = 1
mono_colr = 'k'
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.linewidth'] = box_lw
plt.rcParams['xtick.major.width'] = box_lw
plt.rcParams['ytick.major.width'] = box_lw
'''
Fonts & size
'''
plt_font_size = 8
lgd_font_size = 8
plt.rcParams['font.family'] = "TeX Gyre Termes"
#plt.rc('font', **{'family' : 'sans-serif', 'sans-serif' : ['Myriad Pro']})
plt.rcParams['font.size'] = plt_font_size
plt.rcParams['axes.labelsize'] = plt_font_size
plt.rcParams['axes.titlesize'] = plt_font_size
plt.rcParams['xtick.labelsize'] = plt_font_size
plt.rcParams['ytick.labelsize'] = plt_font_size
plt.rcParams['xtick.major.pad'] = 2
plt.rcParams['ytick.major.pad'] = 2
plt.rcParams['legend.fontsize'] = lgd_font_size
'''
Plots
'''
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markeredgewidth'] = 3
plt.rcParams['errorbar.capsize'] = 5
'''
Colours
'''
plt.rcParams['axes.titlecolor'] = mono_colr
plt.rcParams['axes.edgecolor'] = mono_colr
plt.rcParams['axes.labelcolor'] = mono_colr
plt.rcParams['xtick.color'] = mono_colr
plt.rcParams['xtick.labelcolor'] = mono_colr
plt.rcParams['ytick.color'] = mono_colr
plt.rcParams['ytick.labelcolor'] = mono_colr
'''
LaTeX
'''
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = '\n'.join([
    r'\usepackage[T3,T1]{fontenc}',
    r'\DeclareSymbolFont{tipa}{T3}{cmr}{m}{n}',
    r'\DeclareMathAccent{\invbreve}{\mathalpha}{tipa}{16}',
    r'\usepackage{siunitx}',
    r'\DeclareSIUnit\crewmember{CM}',
    r'\sisetup{range-units=single}',
    r'\sisetup{range-phrase=\textup{--}}'
])  # Preamble must be one line!

In [ ]:
AX_NUM = True
SHOW_LGD = False
mec_colr = 'LimeGreen'
np_colr = 'DodgerBlue'

data_in_path = "./zea-data/"
img_out_path = "./NP-fits/"
data_out_path = "./sens-mat/"

zea_cco2 = 525
zea_ppfd = 225
conditions = np.array([[zea_cco2,zea_ppfd]])  # conditions = np.array([[525, 225], [1200, 500]])
crop_list = ['dry_bean', 'lettuce', 'peanut','rice','soybean','sweet_potato','tomato','wheat','white_potato']

In [ ]:
# For all, we take in c_CO2 and Phi_gamma because N uptake probably depends on them
# Delta mB/Delta t = YN(t) * mN(t)

def run_models(params, cropname, conds,exp_times=None):
    
    #print(params.shape)
    def calc_eta_u(t, crop, c_CO2, Phi_gamma):
        '''
        N uptake performance, 1 = max
        Could account for things like temp, pH
        Assume max for now
        Dimensionless
        '''
        return 1

    def calc_mu_N(t, crop, c_CO2, Phi_gamma):
        '''
        seems to decrease over time
        needs to depend on c_N where high or low c_N limits mu_N
        could measure by mu_N = [ln(m_N(t2)) - ln(m_N(t1))] / (t2 - t1)
        linear fit from Normal N
        units of day^-1
        '''
        mu_N = -params[7] * t + params[6]
        return mu_N

    def calc_eta_N(t, crop, c_CO2, Phi_gamma):
        '''
        amount of plant you get per amount of N over time step
        eta_N = m_B / <m_N>
        can approach zero, but should not become negative
        dimensionless, but g_DW / g_N
        '''
        eta_N = -params[5]*t + params[4] 
        return eta_N

    def calc_m_N(t, crop, c_CO2, Phi_gamma):
        '''
        m_N0 unit: g
        K unit: g
        r unit: day^-1
        m_N unit: g
        '''
        m_N0 = params[0] # estimate from data using N percentage in biomass
        r = params[1] # could change based on N in nutrient solution
        K = params[2]
        alpha = params[3] # corrective term
        m_N = alpha * (m_N0 * K * np.exp(r * t)) / ((K - m_N0) + m_N0 * np.exp(r * t))  # huger guess
        return m_N

    def calc_Y_N(t, crop, c_CO2, Phi_gamma):
        Y_N = calc_eta_u(t, crop, c_CO2, Phi_gamma) * calc_mu_N(t, crop, c_CO2, Phi_gamma) * calc_eta_N(t, crop, c_CO2, Phi_gamma)
        return Y_N

    def calc_m_B_NP(t, crop, c_CO2, Phi_gamma):
        return calc_Y_N(t,crop,c_CO2,Phi_gamma) * calc_m_N(t,crop,c_CO2,Phi_gamma)

    def NP_model(t, y, crop, c_CO2, Phi_gamma):
        Neq = len(y)

        ## Prepare dydt array
        dydt = np.zeros((1, Neq))

        ## Define dydt
        dydt[0, 0] = calc_m_B_NP(t, crop, c_CO2, Phi_gamma)

        return [np.transpose(dydt)]
  
    ## Define directory and locations
    directory = 'parameter-lists/'
    filename = 'crop_parameters_FPSD.xlsx'

    ## Load standard parameters from BVAD
    filename_full = directory + filename
    
    crop_type = cropname
    crop = Crop(crop_type, filename_full=filename_full)
    endtime = crop.t_M
    tspan = [0, endtime]
    if len(exp_times) == 0:
        t_eval = np.arange(0, endtime+1, 1) #Where do we want the solution
    else:
        t_eval = exp_times.reshape(-1,)
    y0 = [0,0,50]
    c_CO2 = conds[0]
    Phi_gamma = conds[1]

    sigma_N = crop.sigma_N
    f_E = crop.f_E

    sol_NP = integrate.solve_ivp(NP_model, tspan, y0, args=(cropname,c_CO2,Phi_gamma), method='LSODA', t_eval=t_eval)
    sol_NP.y[0] = sol_NP.y[0] * sigma_N # need to go from single plant to areal basis; by default NP is per plant

    # Put MEC on top of it
    def mec_model(t, y, crop, CO2, PPF):
        Neq = len(y)

        ## Prepare dydt array
        dydt = np.zeros((1, Neq))

        ## Define dydt
        dydt[0, 0] = calc_m_B(t, crop, CO2, PPF)

        return [np.transpose(dydt)]

    ## Perform integration
    start_time = time.time()
    sol_MEC = integrate.solve_ivp(mec_model, tspan, y0, args=(crop,c_CO2,Phi_gamma), method='LSODA', t_eval=t_eval)

    #plt.plot(sol_MEC.t, sol_MEC.y[0], linewidth=4, color = 'g', ls='--', label="MEC prediction")
    #plt.legend()
    #plt.show()

    # show function values over time
    f=np.zeros((50,1))
    g=np.zeros((50,1))
    h=np.zeros((50,1))
    j=np.zeros((50,1))

    for i in range(0,50):
        f[i] = calc_mu_N(i, crop, c_CO2, Phi_gamma)
        g[i] = calc_m_N(i, crop, c_CO2, Phi_gamma)
        h[i] = calc_Y_N(i, crop, c_CO2, Phi_gamma)
        j[i] = calc_eta_N(i, crop, c_CO2, Phi_gamma)

    #plt.plot(np.arange(0,50), f, label="$\mu_N$")
    #plt.plot(np.arange(0,50), g, label="$m_N$")
    #plt.plot(np.arange(0,50), h, label="$\dot{Y}_N$")
    #plt.plot(np.arange(0,50), j, label="$\eta_N$")
    #plt.legend()
    
    return sol_MEC.t, sol_MEC.y[0], sol_NP.t, sol_NP.y[0]

### Define the fitting function

In [ ]:
def objective_function(x,cropname,conds,print_flag):
    """
    Fitting function to be minimized in order to compare
    MEC and NP
    x:Model parameters
    print_flag : whether to plot predictions or not
    we dont need to see all the plots but it helps visualize 
    """
    t_MEC, y_MEC, t_NP, y_NP = run_models(x,cropname,conds,[])

    if print_flag:
        pltargs = {'alpha': 1, 'lw': 2.5, 'dashes':[2.5, 1]}
        plt.figure(figsize=(2,2))
        plt.plot(t_MEC, y_MEC, label='MEC Model', c=mec_colr, **pltargs)
        plt.plot(t_NP, y_NP, label='NP Model', c=np_colr, alpha=1, zorder=99, lw=1)
        
        # if not AX_NUM: plt.xticks([]); plt.yticks([])
        # axes setup for figure 5
        if cropname == "dry_bean" or cropname == "sweet_potato":
            plt.ylabel(r"$\invbreve{m}_\text{T}\ [\si{\gram\of{DW}\per\meter\squared}]$")

        if cropname == "wheat":
            plt.yticks(np.arange(0,600,150))

        if SHOW_LGD: plt.legend()
            
        plt.savefig(img_out_path + cropname + "_co2-" + str(conds[0]) + "_ppfd-" + str(conds[1]) + "_NP-fit.png", bbox_inches='tight', transparent=True)
        plt.show()

    a = y_MEC
    b = y_NP
    diff = np.abs(a - b) 
    norm = np.linalg.norm(diff, ord=2)
    return norm


In [ ]:
from scipy.optimize import minimize

In [ ]:
len(crop_list)

### This part takes as input some crop and the conditions, and gives back the fitted parameters
### so that the MEC and NP match

In [ ]:
def mec_np_fit(crop,conds):
    print("Fitting for", crop,"and conditions",conds)
    #The nominal parameters - Defined by Kevin
    x_nom = np.array([0.0017, 0.225, 7,0.085,38.0, 0.95,0.7,0.01])
    f_ev = objective_function(x_nom,crop,conds,True) #evaluate the error based on nominal parameters
    print(f_ev)
    ls = 0.1 #Multiplier for lower bound
    us = 10.0 #Multiplier for upper bound

    if crop == "dry_bean":
        x0 = [0.0017, 0.2, 7 ,0.085,38.0, 0.95,0.7,0.01]  # dry bean
    elif crop == "peanut":
        x0 = [0.002, 0.1, 0.50, 0.3, 33, 0.1, 0.9, 0.01]  # peanut
    elif crop == "tomato":
        x0 = [0.0017, 0.225, 2.0, 0.085, 38.0, 0.95, 0.7, 0.01]  # tomato
    elif crop == "rice":
        x0 = [0.00017, 0.1, 1.5, 0.085, 50, 0.7, 0.7, 0.01]  # rice
        # x0 = [0.0017, 0.225, 1.0 ,0.085,38.0, 0.95,0.7,0.01]  # rice
    elif crop == "soybean":
        x0 = [0.0017, 0.35, 2.5 ,0.085,38.0, 0.95,0.7,0.01]  # soybean
    elif crop == "sweet_potato":
        x0 = [0.01, 0.1, 0.2, 0.03, 75, 0.1, 3, 0.01]  # sweet potato
    elif crop == "wheat":
        x0 = [0.0002, 0.2, 0.01, 0.15, 45.0, 0.8, 1.8, 0.01]  # wheat
    elif crop == "white_potato":
        x0 = [0.0017, 0.2, 1.5, 0.06, 50.0, 0.25, 1, 0.01]  # white potato
    else:
        x0 = [0.0017, 0.225, 7.0 ,0.085,38.0, 0.95,0.7,0.01]  # initial guess for the decision variables
    
    bounds = [(x0[0]*ls,x0[0]*us ), (x0[1]*ls,x0[1]*us ), ((x0[2]*ls,x0[2]*us )),
              (x0[3]*ls,x0[3]*us ),(x0[4]*ls,x0[4]*us ),(x0[5]*ls,x0[5]*us ),
             (x0[6]*ls,x0[6]*us ), (x0[7]*0.8,x0[7]*1.2)]  # bounds on the decision variables
    options = {'disp': False}
    result = minimize(objective_function, x0, args=(crop,conds,False) , bounds=bounds,options=options)
    f_ev = objective_function(result.x,crop,conds,True) #Re-evaluate based on optimal values 
    
    return result

### Initialize xfit to hold all results

In [ ]:
xfit = np.zeros((8,len(crop_list),conditions.shape[0]))

### Change cell type to Code to run. This will take a while. 

We fit in total 8 parameters. So the xfit has the fitted parameters 
for all crops (9) and each set of conditions

In [ ]:
xfit.shape

In [ ]:
crop_list

In [ ]:
conditions

In [ ]:
xfit[:,i,0]
# xfit[:,0,1]

### Get parameters and visualize for each crop and each condition
I am using the objective function plot to visualize. Feel free to change colors

### Plot one crop without re-fitting

In [ ]:
#i: 2 corresponds to peanut, etc.

i = 7
crop_index = i
print(crop_list[i])
conditions_index = 0 #conditions set 1
error = objective_function(xfit[:,crop_index,conditions_index],crop_list[crop_index], conditions[conditions_index],print_flag=True)

### Plot all crops without re-fitting

In [ ]:
for i in range(0,len(crop_list)):
    print(crop_list[i])
    error = objective_function(xfit[:,i,conditions_index],crop_list[i], conditions[conditions_index],print_flag=True)

# Generate sensitivity matrices

In [ ]:
# read csv file
NP_df = pd.read_csv(data_in_path + 'compiled_NP_params_225.csv')
# show the first 5 rows
print(NP_df)

In [ ]:
# Drop the 'crop' column
NP_df_params = NP_df.drop('crop', axis=1)
# Convert DataFrame to NumPy array
xfit = NP_df_params.values
# Print the NumPy array
print(xfit.shape)

### Define lower - upper bounds for sensitivity

In [ ]:
lower_bound = xfit * 0.8
upper_bound = xfit * 1.2

### Define the QoI function, here integral, below 

In [ ]:
lower_bound

In [ ]:
def f_sens(x,cropname):
    """
    For each parameter x, 
    get an integral of the curve over time
    This is a representative scalar quantity
    that reflects how the parameters x affect the dynamics 
    in an average sense
    """
    tspan = np.arange(0, 30.05, 0.05)
    conditions = np.array([525, 225])    
    t_MEC, y_MEC, t_NP, y_NP = run_models(x,cropname, conditions,[])
    integral = np.trapz(y_NP, t_NP)
    #plt.plot(t_NP,y_NP,'-',alpha=k/10,linewidth=5)
    return integral

### Change cell type to Code to run. This will take a while. 

In [ ]:
import numpy as np

# Thresholds
E_dry_bean = 100
E_lettuce = 100
E_peanut = 100
E_rice = 100
E_soybean = 100
E_sweet_potato = 100
E_tomato = 100
E_wheat = 100
E_white_potato = 100

# Assign inputs and outputs to individual arrays
dry_bean_in = np.copy(inputs[:,:,0])
dry_bean_out = np.copy(outputs[:,:,0])
mask = dry_bean_out[:,0] > E_dry_bean
dry_bean_in = dry_bean_in[mask]
dry_bean_out = dry_bean_out[mask]

lettuce_in = np.copy(inputs[:,:,1])
lettuce_out = np.copy(outputs[:,:,1])
mask = lettuce_out[:,0] > E_lettuce
lettuce_in = lettuce_in[mask]
lettuce_out = lettuce_out[mask]

peanut_in = np.copy(inputs[:,:,2])
peanut_out = np.copy(outputs[:,:,2])
mask = peanut_out[:,0] > E_peanut
peanut_in = peanut_in[mask]
peanut_out = peanut_out[mask]

rice_in = np.copy(inputs[:,:,3])
rice_out = np.copy(outputs[:,:,3])
mask = rice_out[:,0] > E_rice
rice_in = rice_in[mask]
rice_out = rice_out[mask]

soybean_in = np.copy(inputs[:,:,4])
soybean_out = np.copy(outputs[:,:,4])
mask = soybean_out[:,0] > E_soybean
soybean_in = soybean_in[mask]
soybean_out = soybean_out[mask]

sweet_potato_in = np.copy(inputs[:,:,5])
sweet_potato_out = np.copy(outputs[:,:,5])
mask = sweet_potato_out[:,0] > E_sweet_potato
sweet_potato_in = sweet_potato_in[mask]
sweet_potato_out = sweet_potato_out[mask]

tomato_in = np.copy(inputs[:,:,6])
tomato_out = np.copy(outputs[:,:,6])
mask = tomato_out[:,0] > E_tomato
tomato_in = tomato_in[mask]
tomato_out = tomato_out[mask]

wheat_in = np.copy(inputs[:,:,7])
wheat_out = np.copy(outputs[:,:,7])
mask = wheat_out[:,0] > E_wheat
wheat_in = wheat_in[mask]
wheat_out = wheat_out[mask]

white_potato_in = np.copy(inputs[:,:,8])
white_potato_out = np.copy(outputs[:,:,8])
mask = white_potato_out[:,0] > E_white_potato
white_potato_in = white_potato_in[mask]
white_potato_out = white_potato_out[mask]

In [ ]:
from scipy.io import savemat
# Save arrays to .mat files
savemat(data_out_path + 'dry_bean_matrices.mat', {'matrix1': dry_bean_in, 'matrix2': dry_bean_out})
savemat(data_out_path + 'lettuce_matrices.mat', {'matrix1': lettuce_in, 'matrix2': lettuce_out})
savemat(data_out_path + 'peanut_matrices.mat', {'matrix1': peanut_in, 'matrix2': peanut_out})
savemat(data_out_path + 'rice_matrices.mat', {'matrix1': rice_in, 'matrix2': rice_out})
savemat(data_out_path + 'soybean_matrices.mat', {'matrix1': soybean_in, 'matrix2': soybean_out})
savemat(data_out_path + 'sweet_potato_matrices.mat', {'matrix1': sweet_potato_in, 'matrix2': sweet_potato_out})
savemat(data_out_path + 'tomato_matrices.mat', {'matrix1': tomato_in, 'matrix2': tomato_out})
savemat(data_out_path + 'wheat_matrices.mat', {'matrix1': wheat_in, 'matrix2': wheat_out})
savemat(data_out_path + 'white_potato_matrices.mat', {'matrix1': white_potato_in, 'matrix2': white_potato_out})

### At this point, you have to process the .mat files with the MATLAB script

# Load saved data and plot it

## Plot from NP params file
* This file was manually compiled by KY

In [ ]:
df = pd.read_csv(data_in_path + 'compiled_NP_params_225.csv')
crop_index = 0
conditions_index = 0 #conditions set 1
for crop_index in range(0,9):
    print(crop_list[crop_index])
    crop_np_params = df.iloc[crop_index,1:].to_numpy()
    error = objective_function(crop_np_params,crop_list[crop_index], conditions[conditions_index],print_flag=True)

## plot sensitivities from files

In [ ]:
import scipy.io as sio

# DRAW_X_TICK = True
# DRAW_Y_TICK = False

crop_list = ['dry_bean', 'lettuce', 'peanut','rice','soybean','sweet_potato','tomato','wheat','white_potato']

for i in range(len(crop_list)):
    print(crop_list[i])
    
    mat_file = data_out_path + crop_list[i] + "_sensitivity_results.mat"
    mat_colname = "sensitivities"
    mat_contents = sio.loadmat(mat_file)
    s_array = mat_contents[mat_colname].flatten()
    
    # Create a list of labels for the plot
    # labels = ['$x_0$', '$x_1$', '$x_2$', '$x_3$', '$x_4$', '$x_5$', '$x_6$']
    labels = [r"$m_\text{N0}$", '$r$', '$K$', r'$\alpha$', r'$\eta_\text{N} (b)$', r'$\eta_\text{N} (m)$', r'$\mu_\text{N} (b)$', r'$\mu_\text{N} (m)$']
    
    # Create the bar plot with black border
    fig, ax = plt.subplots(figsize=(2,2))
    ax.barh(labels, s_array, edgecolor='black')
    
    # # Add labels to the plot
    # ax.set_xlabel('Sensitivity Index Value')
    #ax.set_title('Sensitivity Indices')
    # ax.set_ylabel('Parameter')
    # from matplotlib.ticker import (MultipleLocator)
    # ax.xaxis.set_major_locator(MultipleLocator(0.1))
    # ax.xaxis.set_minor_locator(MultipleLocator(0.05))
    ax.set_xlim(0,0.3)
    
    # if not DRAW_X_TICK: plt.xticks([]);
    # if not DRAW_Y_TICK: plt.yticks([])
    if crop_list[i] != "dry_bean" and crop_list[i] != "sweet_potato":
        plt.yticks([])
    
    # plt.gca().axes.xaxis.set_ticklabels([])
    plt.gca().invert_yaxis()
    
    plt.savefig(img_out_path + crop_list[i] + "-sensitivity.png", bbox_inches='tight', transparent=True)
    
    # # Show the plot
    plt.show()